In [2]:
import numpy as np
import pandas as pd
import os
import sys

In [61]:
res_folder = "./disagreement_matrix/engineering_version/auc_results_diff_classifier/"

In [62]:
auc_results = dict()
for root, path, files in os.walk(res_folder):
    for file in files:
        if file.endswith("xlsx"):
            auc_results[file[:-5]] = pd.read_excel(root+file, index_col=0)

In [63]:
# key as comb, value is fname
surpass_record = dict()

for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    good_records = df_.combination[df_.auc>best_base_auc]
    if len(good_records)>0:
        for m in good_records.values:
            if m in surpass_record.keys():
                surpass_record[m].add(fname)
            else:
                surpass_record[m] = set([fname])

In [64]:
res_list = []
for key, value in surpass_record.items():
    res_list.append((key, len(value)))

In [65]:
all_files_record = {"best_base_auc":[], "ensemble_best_auc":[]}
fnames = []
for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    merge_best = np.max(df_.auc[df_.combination!="best_base_auc"].values)
    all_files_record['best_base_auc'].append(best_base_auc)
    all_files_record['ensemble_best_auc'].append(merge_best)
    fnames.append(fname)
    
all_files_record['DataSets'] = fnames

In [66]:
df_res = pd.DataFrame.from_dict(all_files_record)

In [68]:
pd.DataFrame(sorted(res_list,key=lambda x: 
                    x[1],reverse=True),columns=['ensemble_methods',
                    'num_surpass']).to_excel("./mixed_base_detectors_ensemble.xlsx")

In [69]:
df_res['best_base_auc'] = np.round(df_res['best_base_auc'],3)
df_res['ensemble_best_auc'] = np.round(df_res['ensemble_best_auc'],3)

In [70]:
df_res['diff'] = df_res['ensemble_best_auc'] - df_res['best_base_auc']

In [71]:
q = df_res.loc[:,['ensemble_best_auc','best_base_auc','diff','DataSets']]

In [72]:
q

,ensemble_best_auc,best_base_auc,diff,DataSets
0,0.535,0.545,-0.010,WPBC_withoutdupl_norm_auc_collection
1,0.860,0.850,0.010,mnist_auc_collection
2,0.960,0.960,0.000,PenDigits_withoutdupl_norm_v01_auc_collection
3,0.912,0.904,0.008,PageBlocks_withoutdupl_09_auc_collection
4,0.992,0.992,0.000,KDDCup99_withoutdupl_catremoved_auc_collection
5,0.939,0.935,0.004,cover_auc_collection
6,0.803,0.804,-0.001,Hepatitis_withoutdupl_16_auc_collection
7,0.990,0.990,0.000,WBC_v01_auc_collection
8,0.776,0.777,-0.001,Wilt_withoutdupl_05_auc_collection
9,0.745,0.746,-0.001,HeartDisease_withoutdupl_44_auc_collection


In [73]:
q.to_excel("./MIXED_base_overall_perform.xlsx")